In [1]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
%cd $module_path

/home/efrem/dev/heatflask/backend/heatflask


In [2]:
# %load Users.py
"""
***  For Jupyter notebook ***
Paste one of these Jupyter magic directives to the top of a cell
 and run it, to do these things:
    %%cython --annotate       # Compile and run the cell
    %load Users.py            # Load Users.py file into this (empty) cell
    %%writefile Users.py      # Write the contents of this cell to Users.py
"""

from logging import getLogger
import datetime

import DataAPIs
import Utility

log = getLogger(__name__)
log.propagate = True

COLLECTION_NAME = "users"

# Drop a user after a year of inactivity
MONGO_TTL = 365 * 24 * 3600

ADMIN = [15972102]


class Box:
    collection = None


myBox = Box()


async def get_collection():
    if myBox.collection is None:
        myBox.collection = await DataAPIs.init_collection(
            COLLECTION_NAME, ttl=MONGO_TTL
        )
    return myBox.collection


def mongo_doc(
    # From Strava Athlete record
    id=None,
    username=None,
    firstname=None,
    lastname=None,
    profile_medium=None,
    profile=None,
    measurement_preference=None,
    city=None,
    state=None,
    country=None,
    email=None,
    # my additions
    _id=None,
    ts=None,
    auth=None,
    access_count=None,
    private=None,
    **extras
):
    if not (id or _id):
        log.error("cannot create user with no id")
        return

    return Utility.cleandict(
        {
            "_id": int(_id or id),
            "username": username,
            "firstname": firstname,
            "lastname": lastname,
            "profile": profile_medium or profile,
            "units": measurement_preference,
            "city": city,
            "state": state,
            "country": country,
            "email": email,
            #
            "ts": ts,
            "access_count": access_count,
            "auth": auth,
            "private": private,
        }
    )


async def add_or_update(update_ts=False, inc_access_count=False, **userdict):
    users = await get_collection()
    doc = mongo_doc(**userdict)
    if not doc:
        log.exception("error adding/updating user: %s", doc)
        return

    user_id = doc.pop("_id")

    if update_ts:
        doc["ts"] = datetime.datetime.utcnow()

    updates = {"$set": doc} if doc else {}

    if inc_access_count:
        updates["$inc"] = {"access_count": 1}

    log.debug("calling mongodb update_one with updates %s", updates)

    # Creates a new user or updates an existing user (with the same id)
    try:
        return await users.update_one({"_id": user_id}, updates, upsert=True)
    except Exception:
        log.exception("error adding/updating user: %s", doc)


async def get(user_id):
    users = await get_collection()
    uid = int(user_id)
    query = {"_id": uid}
    try:
        doc = await users.find_one(query)
    except Exception:
        log.exception("Failed mongodb query: %s", query)
        doc = None
    return doc


# Returns an async iterator
async def get_all():
    users = await get_collection()
    return users.find()


async def delete(user_id):
    users = await get_collection()
    uid = int(user_id)
    try:
        return await users.delete_one({"_id": uid})

    except Exception:
        log.exception("error deleting user %d", uid)


def stats():
    return DataAPIs.stats(COLLECTION_NAME)


def drop():
    return DataAPIs.drop(COLLECTION_NAME)


In [3]:
import logging
logging.basicConfig(level="DEBUG")

await DataAPIs.connect()

new_user_dict = {"username": "guy"}
await add_or_update(**new_user_dict)  # should give an error

new_user_dict = {**new_user_dict, "id": 222}
await add_or_update(**new_user_dict)

INFO:DataAPIs:Connected to MongoDB and Redis
ERROR:__main__:cannot create user with no id
ERROR:__main__:error adding/updating user: None
NoneType: None
DEBUG:__main__:calling mongodb update_one with updates {'$set': {'username': 'guy'}}


In [4]:
await get(222)

{'_id': 222, 'username': 'guy'}

In [5]:
await add_or_update(_id=222, private=False)

await add_or_update(id=222, inc_access_count=True)

result = await add_or_update(id=222, update_ts=True)
user_info = await get(222)
result.raw_result, user_info

DEBUG:__main__:calling mongodb update_one with updates {'$set': {'private': False}}
DEBUG:__main__:calling mongodb update_one with updates {'$inc': {'access_count': 1}}
DEBUG:__main__:calling mongodb update_one with updates {'$set': {'ts': datetime.datetime(2022, 2, 20, 6, 23, 14, 132804)}}


({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True},
 {'_id': 222,
  'username': 'guy',
  'private': False,
  'access_count': 1,
  'ts': datetime.datetime(2022, 2, 20, 6, 23, 14, 132000)})

In [6]:
result = await delete(222)
result.raw_result

{'n': 1, 'ok': 1.0}

In [7]:
# import real user data from Strava
import Strava
print("Paste this URL into your browser and retrieve the code:\n", Strava.auth_url())

Paste this URL into your browser and retrieve the code:
 https://www.strava.com/oauth/authorize?client_id=12700&response_type=code&approval_prompt=force&scope=read,activity:read,activity:read_all&redirect_uri=http:%2F%2Flocalhost%2Fexchange_token


In [8]:
# exchange code for token
CODE = "7eb4e6a3f74ef50947f39bda8660ba3400d2723b"
C = Strava.AsyncClient("admin")
new_info = await C.update_access_token(code=CODE)
user_dict = new_info.pop("athlete")
new_info, user_dict

DEBUG:Strava:refreshing access token from code


AttributeError: 'NoneType' object has no attribute 'pop'

In [ ]:
mongo_doc(**user_dict)

In [ ]:
await add_or_update(
    auth=new_info,
     update_ts=True, 
    inc_access_count=True,
    **user_dict, 
)

In [61]:
# async_cursor = await get_all()
# userslist1 = [user async for user in async_cursor]  

async_cursor = await get_all()
userslist2 = await async_cursor.to_list(length=None)
userslist2

[{'_id': 15972102,
  'access_count': 1,
  'auth': {'expires_at': 1644627379,
   'refresh_token': '05867993a2d0c5b60c51653636a9c295348551f3',
   'access_token': 'ebc458c349e2efd55f42447781699f393ef16743'},
  'city': 'Oakland',
  'country': 'United States',
  'firstname': '👣',
  'lastname': 'Efrem',
  'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/medium.jpg',
  'state': 'California',
  'ts': datetime.datetime(2022, 2, 11, 18, 56, 28, 793000),
  'username': 'bfef'},
 {'_id': 97298375,
  'firstname': 'Jake',
  'lastname': 'Wilson',
  'profile': 'https://lh3.googleusercontent.com/a/AATXAJyxErxQdcnUs1bpEpunMeWJEwdDtom8YzZ9Jyth=s96-c',
  'access_count': 0,
  'auth': {'access_token': '8c500838353384d7e84a2fef97520eb4b3108eec',
   'refresh_token': '7e20e49d2c5623b0068fe1ba61deb3f252dbea1a',
   'expires_at': 1641615688},
  'private': True},
 {'_id': 3045216,
  'username': 'hheinermann',
  'firstname': 'Hans',
  'lastname': 'Heinermann',
  'profile': 'https

In [22]:
# Import legacy Users database
import os
pgurl = os.environ["REMOTE_POSTGRES_URL"]
from sqlalchemy import create_engine, text

results = None
with create_engine(pgurl).connect() as conn:
    result = conn.execute(text("select * from users"))
results = result.all()

In [41]:
results[2000]

(27707458, 'julian_starke', 'Julian', 'Starke', 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/27707458/11687913/4/medium.jpg', '{"access_token": "65089094fb8768aecb3b498524e6abf16003b647", "refresh_token": "9256493931c072fe538ee2ec87f6590300b00a73", "expires_at": 1586709840}', None, 'Berlin', 'Berlin', 'Deutschland', None, datetime.datetime(2019, 12, 25, 11, 38, 41, 513618), 28, False, None)

In [59]:
import json
from datetime import datetime
docs = []

for (
    id,
    username,
    firstname,
    lastname,
    profile,
    access_token,
    measurement_preference,
    city,
    state,
    country,
    email,
    dt_last_active,
#     dt_indexed,
    app_activity_count,
    share_profile,
    xxx
) in results:
    if (id in ADMIN) or (dt_last_active is None):
        log.info("skipping %d", id)
        continue
    try:
        docs.append(mongo_doc(
            # From Strava Athlete record
            id=id,
            username=username,
            firstname=firstname,
            lastname=lastname,
            profile=profile,
            measurement_preference=measurement_preference,
            city=city,
            state=state,
            country=country,
            email=email,
            #
            ts=dt_last_active,
            auth=json.loads(access_token),
            access_count=app_activity_count,
            private=not share_profile,
        ))
    except json.JSONDecodeError:
        pass
        
ids = [u["_id"] for u in docs]

INFO:__main__:skipping 97298375
INFO:__main__:skipping 6851964
INFO:__main__:skipping 15972102


In [55]:
docs[323], ids

({'_id': 706357,
  'username': 'paul_cope',
  'firstname': 'Paul',
  'lastname': 'Cope',
  'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/706357/469668/1/medium.jpg',
  'city': 'Molesey',
  'state': 'England',
  'country': 'United Kingdom',
  'ts': datetime.datetime(2020, 3, 10, 13, 46, 41, 314388),
  'access_count': 3,
  'auth': {'access_token': '31e7fb2836e849f2136d396295288d7fb5f4c38a',
   'refresh_token': '2e057e737c51f42d51b5d9e8757768c8ba302e04',
   'expires_at': 1586903229},
  'private': True},
 [3045216,
  37634167,
  14569586,
  7222491,
  1667371,
  6483336,
  10007779,
  2276263,
  5938463,
  322591,
  3054842,
  20829583,
  3380054,
  15050965,
  63192086,
  14455396,
  21161919,
  12022644,
  47638298,
  3897840,
  42079695,
  44578106,
  44251926,
  50391376,
  15512586,
  7154714,
  63041546,
  29310238,
  43437525,
  8023794,
  4723100,
  16093897,
  23776025,
  19849443,
  4930254,
  8246757,
  47322987,
  11669428,
  93117826,
  2620260,
  8784402,

In [57]:
[d.get("ts") for d in docs]

[datetime.datetime(2019, 12, 18, 11, 34, 30, 32849),
 datetime.datetime(2019, 11, 20, 19, 27, 49, 800961),
 datetime.datetime(2020, 11, 29, 12, 58, 42, 531564),
 datetime.datetime(2020, 8, 12, 19, 34, 23, 188812),
 datetime.datetime(2020, 5, 23, 21, 0, 22, 793109),
 datetime.datetime(2020, 2, 28, 8, 37, 4, 762431),
 datetime.datetime(2020, 5, 2, 8, 36, 40, 648286),
 datetime.datetime(2020, 7, 26, 6, 44, 38, 338271),
 datetime.datetime(2020, 8, 19, 21, 30, 0, 205298),
 datetime.datetime(2020, 6, 11, 11, 42, 33, 572094),
 datetime.datetime(2020, 4, 23, 1, 36, 21, 103092),
 datetime.datetime(2020, 8, 17, 17, 51, 38, 530182),
 datetime.datetime(2021, 4, 27, 18, 10, 15, 834646),
 datetime.datetime(2021, 3, 17, 6, 50, 11, 442520),
 datetime.datetime(2020, 8, 23, 15, 49, 57, 396099),
 datetime.datetime(2020, 6, 24, 22, 46, 59, 788185),
 datetime.datetime(2021, 12, 30, 19, 21, 17, 172043),
 datetime.datetime(2021, 3, 13, 20, 27, 1, 528336),
 datetime.datetime(2021, 8, 26, 1, 11, 23, 707411),
 

In [60]:
users = await get_collection()
await users.delete_many({"_id": {"$in": ids}})
await users.insert_many(docs)

In [31]:
await DataAPIs.disconnect()

INFO:DataAPIs:Disconnected from MongoDB and Redis
